In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\Irshad\Downloads\csvFile660.csv")  #Directory of csv file downoaded from India WRIS
df = df.reset_index()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df

,Dates,Last 10 Year Average (Flow in cumecs),Last Year (Flow in cumecs),Current Year (Flow in cumecs),Level (m)
1,01-Jan-1989,23.0858,19.83,12,219.91
2,02-Jan-1989,22.506,20.83,11.45,219.88
3,03-Jan-1989,23.531000000000002,20.5,11.22,219.88
4,04-Jan-1989,23.376,20.61,11.65,219.88
5,05-Jan-1989,22.72,19.04,11.9,219.88
...,...,...,...,...,...
10694,27-Dec-2018,4.897104520000001,3.2347429,7.8896513,222.555
10695,28-Dec-2018,4.62958966,3.4784873,7.7206583,222.55
10696,29-Dec-2018,5.7335586,3.430631,7.626921,222.545
10697,30-Dec-2018,4.89127535,2.6498485,8.079,222.545


In [3]:
df = df.drop(columns=['Last 10 Year Average (Flow in cumecs)','Last Year (Flow in cumecs)'])
df['Dates'] = pd.to_datetime(df['Dates'], format="%d-%b-%Y")
df = df.set_index('Dates')
df_numeric = df.apply(pd.to_numeric, errors='coerce')
df = df_numeric.dropna()

In [4]:
df_filled = df.asfreq('D')
na_count = df_filled["Current Year (Flow in cumecs)"].isna().sum()
print("Number of missing values: ",na_count)

Number of missing values:  263


In [5]:
start_date = df_filled.index.min()
end_date = df_filled.index.max()

In [6]:
df_filled["day"] = df_filled.index
df_filled['Day_Month'] = df_filled['day'].dt.strftime('%m-%d')
df_filled = df_filled.drop(columns=['day'])

In [7]:
daily_average = df_filled.groupby('Day_Month').mean()
df_filled = df_filled.set_index("Day_Month")

In [8]:
df_filled['Current Year (Flow in cumecs)'] = df_filled['Current Year (Flow in cumecs)'].fillna(daily_average['Current Year (Flow in cumecs)'])
df_filled['Level (m)'] = df_filled['Level (m)'].fillna(daily_average['Level (m)'])
date_range = pd.date_range(start=start_date, end=end_date)
df_filled.index = date_range
df_filled

,Current Year (Flow in cumecs),Level (m)
1989-01-01,12.000000,219.910
1989-01-02,11.450000,219.880
1989-01-03,11.220000,219.880
1989-01-04,11.650000,219.880
1989-01-05,11.900000,219.880
...,...,...
2018-12-27,7.889651,222.555
2018-12-28,7.720658,222.550
2018-12-29,7.626921,222.545
2018-12-30,8.079000,222.545


In [9]:
df_filled.reset_index().rename(columns={'index': 'Date'}).to_csv('Jodhra_Filled_streamflow_data.csv', index=False)